# INF200 Lecture No J05
### Hans Ekkehard Plesser / NMBU
### 14 January 2019

## Today's topics
- Repository access
- Deleting from lists
- Visualization
    - Plotting in PyCharm
    - Plotting in Python scripts and the GIL
    - Updating data in plots
    - A time counter in a figure
    - An island map
    - Adding sliders and buttons
    - RandVis: Making movies
- Model dynamics (separate notebook)

## Repository access

- Please give the following Bitbucket users access to your group repo
    - heplesser
    - daniel_ambrosius_prince
    - eivinsve

## Deleting from lists

- Removing elements from a list inside a loop over the list is dangerous
- It can confuse the list iteration

### A correct loop

In [1]:
d = list(range(10))
for n in d:
    print('Testing', n)
    if n % 2 == 0 or n % 3 == 0:
        print('    divisible')

Testing 0
    divisible
Testing 1
Testing 2
    divisible
Testing 3
    divisible
Testing 4
    divisible
Testing 5
Testing 6
    divisible
Testing 7
Testing 8
    divisible
Testing 9
    divisible


### A confused loop

In [2]:
d = list(range(10))
for n in d:
    print('Testing', n)
    if n % 2 == 0 or n % 3 == 0:
        d.remove(n)
print(d)

Testing 0
Testing 2
Testing 4
Testing 6
Testing 8
[1, 3, 5, 7, 9]


### A better solution: keep the good ones

In [3]:
d = list(range(10))
d = [n for n in d if not (n % 2 == 0 or n % 3 == 0)]
print(d)

[1, 5, 7]


This is called filtering and is an essential part of functional programming languages.

### Alternative solution: Create list of numbers to delete

In [4]:
d = list(range(10))
should_delete = []
for n in d:
    print('Testing', n)
    if n % 2 == 0 or n % 3 == 0:
        should_delete.append(n)
for el in should_delete:
    d.remove(el)
print(d)

Testing 0
Testing 1
Testing 2
Testing 3
Testing 4
Testing 5
Testing 6
Testing 7
Testing 8
Testing 9
[1, 5, 7]


## Visualization

### Plotting in PyCharm

- Newer versions of PyCharm have a "Scientific" mode that supports plotting inside the PyCharm window
- That does not work well with this project
- To turn it off, go to `Preferences > Tools > Python Scientific` and remove the check mark for `Show plots in tool window`

### Plotting in Python scripts and the GIL

- Parallel programs
    - A program may run internally on multiple *threads*
    - *Threads* are essentially multiple "workers" inside a single program working in parallel
    - Problem: Two workers may change the same data at the same time: Chaos!
    - Solution in CPython: The GIL
- GIL: Global Interpreter Lock
    - Present in CPython and several other Python implementations
    - In each Python program, there is *exactly one* GIL (think of it as a baton,  "stafettpinne")
    - Only the thread holding the GIL can work, all others are paused
- Plotting and the GIL
    - Usually, a figure window is managed by its own thread
    - The thread can only draw to the screen while it has the GIL
    - In a loop like (`plt` is `matplotlib.pyplot`)
        ```
        for alpha in range(100):
            plt.plot(x, sin(alpha*x))
        ```
      the graphics thread may never get access to the GIL
    - We must *pause* the main thread to ensure figure updating
        ```
        for alpha in range(100):
            plt.plot(x, sin(alpha*x))
            plt.pause(1e-6)
        ```
    - Pause duration is in seconds and can be very short
- Other (more or less) relevant plot commands
    - `plt.show()` hands over the GIL to the figure thread, the main thread is frozen until the figure window is closed; usually to be used as the last line in a script, so the user can look at the figure(s) generated
    - `plt.ion()` ("interactive on") ensures that figures are updated automatically after each `plt` command (e.g. change in axis limits); meaningful when working interactively in an IPython shell
    - `plt.ioff()` turns interactive mode off again
    - `plt.draw()` updates figure; when not in interactive mode, the effect of many `plt` commands will become visible only upon `plt.draw()`
- Detailed behavior can depend on operating system, Matplotlib backend and the exact way the Python script is run
- References:
    - [Python Wiki on GIL](https://wiki.python.org/moin/GlobalInterpreterLock)
    - [Wikipedia article on GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock)
    - [David Beazly: Understanding GIL](http://www.dabeaz.com/python/UnderstandingGIL.pdf)

### Updating data in plots

- Example program: `Plotting/plot_update.py`
    - Creates list of random numbers, updates plot every time a new number is added
    - Note the slow-down when running function `replot(1000)`
    - Then run just function `update(1000)`: no slow-down
- Plotting using `Figure` and `Axes` objects
    - Plotting using `plt....()` functions is easy, but limits our possibilities
    - By using the equivalent methods on `Figure`, `Axes`, `Line`, etc objects, we get much more control
- Reason:
    - `plt.plot()` adds new coordinate system (`Artist`) to figure each time it is called
    - Matplotlib must render all `Artist`s each time the plot is updated
- Solution:
    - Call `plt.plot()` once, afterwards just update the values
    
        ```python       
        def update(n_steps):
            fig = plt.figure()
            ax = fig.add_subplot(1, 1, 1)
            ax.set_xlim(0, n_steps)
            ax.set_ylim(0, 1)

            line = ax.plot(np.arange(n_steps),
                           np.full(n_steps, np.nan), 'b-')[0]

            for n in range(n_steps):
                ydata = line.get_ydata()
                ydata[n] = np.random.random()
                line.set_ydata(ydata)
                plt.pause(1e-6)
        ```

    - Notes:
        - We start with a line having as many data points as we want to have in the end
        - We set all y-values to `np.nan` (not-a-number), a special value that is *not plotted*
        - We must set x and y axis limits before we plot
        - `line.set_ydata(ydata)` updates the data in the figure

### A time counter in a figure

- Example program: `Plotting/time_counter.py`
    - Creates several axes for plotting (not actually used in this example)
    - Adds one axes at prescribed location, turns coordinate axis off
    - Adds text
    - Updates text in loop
    
        ```python
        fig = plt.figure()

        # normal subplots
        ax1 = fig.add_subplot(2, 3, 1)
        ax2 = fig.add_subplot(2, 3, 3)
        ax3 = fig.add_subplot(2, 3, 4)
        ax4 = fig.add_subplot(2, 3, 5)
        ax5 = fig.add_subplot(2, 3, 6)

        # axes for text
        axt = fig.add_axes([0.4, 0.8, 0.2, 0.2])  # llx, lly, w, h
        axt.axis('off')  # turn off coordinate system

        template = 'Count: {:5}'
        txt = axt.text(0.5, 0.5, template.format(0),
                       horizontalalignment='center',
                       verticalalignment='center',
                       transform=axt.transAxes)  # relative coordinates

        plt.pause(1e-6)  # pause required to make figure visible

        input('Press ENTER to begin counting')

        for k in range(40):
            txt.set_text(template.format(k))
            plt.pause(0.1)  # longer pause so we see timer running
        ```
        
- For more fine-grained control of subplot placement, see Matplotlib's `GridSpec` (https://matplotlib.org/users/gridspec.html)
- You can also place text directly into a figure; see Matplotlib Gallery for examples (https://matplotlib.org/gallery.html#text_labels_and_annotations)

### An island map

- Example program: `Plotting/mapping.py`
    - Starts from map given as multiline string
    - Creates nested list, where each cell on island is represented by RGB color triplet
        - Three nesting levels: row - column - color component
    - Draws displays map using `imshow()`
    - Manually adds legend by adding `Rectangle` patches

### Adding sliders and buttons

- See the example programs from the lecture on 26 November 2018 as starting points
- When slider value is changed, make necessary `biosim.set_..._parameters()` call
- When `Interrupt` button is pressed, the event handler must set a flag (boolean variable) that is checked by the main simulation loop at the end of each year

### RandVis: Making movies

#### Animations

- Matplotlib has built-in animation support
    - Useful for interactive animation on the screen
    - Can also create movies
    - See, e.g., the [Matplotlib Animation Tutorial](https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/) by Jake Vanderplas
- Do it yourself
    - We can do the same thing by regularly updating figures, as shown above
        
#### Making movies from animations

- Principles
    - Movies are sequences of images
    - Write all images for a movie to file first
    - Use *external application* to combine images to movie
- With `matplotlib.animation`
    - Write movie by calling `save()` on `Animation` object
    - Removes temporary files automatically when movie is ready
    - To see available movie writers (movie file formats):
    
            import matplotlib.animation as animation
            print animation.writers.list()
            
- Manually
    - Write figures to file after updating with `savefig()`
    - Usual image format: PNG (Portable network graphics)
    - Enumerate files from 0, with leading digits:
            
            img_00000.png
            img_00001.png
            
    - Use external application to combine figures to movie
    
#### External applications to convert images to movies

##### Animated GIF
- Very robust (plays anywhere), but file may be very large
- Use [ImageMagick](http://www.imagemagick.org)
    - For OSX and Win, you can download installation package
    - For OSX, you can also install using `brew install ImageMagick`
    - Package also supports all kinds of image file format transformations and image manipulations

##### Video formats such as MP4, AVI, ...
- Typically much smaller files
- High levels of compression may lead to artefacts
- Not as robust, if you choose unsuitable parameters, the resulting file may not play on some devices or player programs
- Use [FFMPEG](https://www.ffmpeg.org)
- For a versatile player, see [VLC](https://www.videolan.org/vlc/)
        
#### Example of simulation with manual movie production

- `RandVis` package under `Project/SampleProjects` in course repository

### DO NOT COMMIT IMAGE OR MOVIE FILES TO YOUR REPOSITORY!!!

- Files will take a lot of place
- Re-running your simulations will create different files even if the simulations are the same
- Files are compressed binary files and cannot be compared/diff'ed meaningfully
    - Requires a lot of space
- **Make sure Git ignores directories with graphics!**
